# REST-1 ip주소에서 지역정보 알아내기

In [1]:
import json
import urllib

def getCountry(ipAddress):
    response = urllib.urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("39.118.87.152"))

KR


In [2]:
import requests
send_url = 'http://freegeoip.net/json/39.118.87.152'
r = requests.get(send_url)

In [3]:
j=json.loads(r.text)

In [4]:
type(j)
print j.keys()
print j['city']

[u'city', u'region_code', u'region_name', u'ip', u'time_zone', u'longitude', u'metro_code', u'latitude', u'country_code', u'country_name', u'zip_code']
Seoul


In [5]:
for k,v in j.iteritems():
    print k,"\t: ",v

city 	:  Seoul
region_code 	:  11
region_name 	:  Seoul
ip 	:  39.118.87.152
time_zone 	:  Asia/Seoul
longitude 	:  126.9743
metro_code 	:  0
latitude 	:  37.5111
country_code 	:  KR
country_name 	:  Republic of Korea
zip_code 	:  


# REST-2 키를 저장하고 읽기

In [6]:
def getKey(keyPath):
    d=dict()
    f=open(keyPath,'r')
    for line in f.readlines():
        row=line.split('=')
        row0=row[0]
        d[row0]=row[1].strip()
    return d

In [7]:
import os

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=getKey(keyPath)

In [8]:
print key

{'dataseoul': '4d4242484864617938315369514567', 'gokr': 'lbJPWb7U14QILqLdfz6sDx9MOQA9qogxZ0x5BjiNCpH5wd5qMQtyW%2Bn1EmzsI1c9tzS7edrRiHAKZeVxTPAwFQ%3D%3D'}


# REST-3 서울시 1호선 지하철역 목록 구하기

In [11]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='SearchSTNBySubwayLineService'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
print params[31:]

xml\SearchSTNBySubwayLineService\1\10\2


In [12]:
import urlparse
_url='http://openAPI.seoul.go.kr:8088/'
url=urlparse.urljoin(_url,params)

print url

http://openAPI.seoul.go.kr:8088/4d4242484864617938315369514567\xml\SearchSTNBySubwayLineService\1\10\2


In [13]:
myurl='http://openapi.seoul.go.kr:8088/4d4242484864617938315369514567/xml/SearchSTNBySubwayLineService/1/20/1/'
import requests
data=requests.get(myurl)

In [14]:
import re
p=re.compile('<STATION_NM>(.+?)</STATION_NM>')
res=p.findall(data.text)
for item in res:
    print item

소요산
동두천
보산
동두천중앙
지행
덕정
덕계
양주
녹양
가능
의정부
회룡
망월사
도봉산
도봉
방학
창동
녹천
월계
광운대


# REST-4 서울시 외국인 주민 자녀 국적취득 년도별 시군구 합계 구하기

In [15]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='ListLocaldata470401S'
START_INDEX=str(1)
END_INDEX=str(5)

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX)

print params[31:]

xml\ListLocaldata470401S\1\5


In [16]:
import urlparse
_url='http://openapi.seoul.go.kr:8088'
url=urlparse.urljoin(_url,params)

print url

http://openapi.seoul.go.kr:8088/4d4242484864617938315369514567\xml\ListLocaldata470401S\1\5


In [17]:
myurl='http://openapi.seoul.go.kr:8088/4d4242484864617938315369514567/xml/ListLocaldata470401S/1/5'
import requests
data=requests.get(myurl).text
print data

<?xml version="1.0" encoding="UTF-8"?><ListLocaldata470401S>
<list_total_count>319</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<SITUYM>2013</SITUYM>
<CGGNM>서울특별시 종로구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>462</STATMAN>
</row>
<row>
<SITUYM>2014</SITUYM>
<CGGNM>서울특별시 종로구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>468</STATMAN>
</row>
<row>
<SITUYM>2013</SITUYM>
<CGGNM>서울특별시 중구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 19세 이상</KDDETAILSENM>
<STATMAN>232</STATMAN>
</row>
<row>
<SITUYM>2013</SITUYM>
<CGGNM>서울특별시 광진구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 18세 이하</KDDETAILSENM>
<STATMAN>1275</STATMAN>
</row>
<row>
<SITUYM>2014</SITUYM>
<CGGNM>서울특별시 광진구</CGGNM>
<KDDETAILSENM>국적취득한 자녀 만 19세 이상</KDDETAILSENM>
<STATMAN>536</STATMAN>
</row>
</ListLocaldata470401S>



In [18]:
print type(data.encode('utf-8'))

<type 'str'>


In [19]:
import lxml
import lxml.etree
import StringIO

tree=lxml.etree.fromstring(data.encode('utf-8'))

nodes=tree.xpath('//STATMAN')
for node in nodes:
    print node.text

462
468
232
1275
536


# REST-5 2015년 서울시 지하철역별 월별 승하차인원 구하기

In [29]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='CardSubwayStatisticsService'
START_INDEX=str(1)
END_INDEX=str(5)
USE_MON='201306'

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,USE_MON)

print params[31:]

xml\CardSubwayStatisticsService\1\5\201306


In [30]:
import urlparse
_url='http://openApi.seoul.go.kr:8088'
url=urlparse.urljoin(_url,params)

print url

http://openApi.seoul.go.kr:8088/4d4242484864617938315369514567\xml\CardSubwayStatisticsService\1\5\201306


In [22]:
myurl='http://openAPI.seoul.go.kr:8088/4d4242484864617938315369514567/xml/CardSubwayStatisticsServie/1/5/201306'
import requests
data=requests.get(myurl).text
print data

<?xml version="1.0" encoding="UTF-8"?><RESULT>
<CODE>ERROR-500</CODE>
<MESSAGE>서버 오류입니다.\n지속적으로 발생시 열린 데이터 광장으로 문의(Q&amp;A) 바랍니다.</MESSAGE>
</RESULT>



# REST-6 2013년 1년 동안 지하철역 승하차 인원

In [32]:
KEY=str(key['dataseoul'])
TYPE='xml'
SERVICE='CardSubwayStatisticsService'
START_INDEX=str(1)
END_INDEX=str(5)
USE_MON='201306'

params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,USE_MON)

print params[31:]

xml\CardSubwayStatisticsService\1\5\201306


In [33]:
import urlparse
_url='http://openApi.seoul.go.kr:8088'
url=urlparse.urljoin(_url,params)

print url

http://openApi.seoul.go.kr:8088/4d4242484864617938315369514567\xml\CardSubwayStatisticsService\1\5\201306


In [36]:
myurl='http://openApi.seoul.go.kr:8088/4d4242484864617938315369514567/xml/CardSubwayStatisticsService/1/5/201306'
data=requests.get(myurl).text
print data

<?xml version="1.0" encoding="UTF-8"?><CardSubwayStatisticsService>
<list_total_count>530</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>용문</SUB_STA_NM>
<RIDE_PASGR_NUM>111275</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>108878</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>원덕</SUB_STA_NM>
<RIDE_PASGR_NUM>11495</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10964</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>양평</SUB_STA_NM>
<RIDE_PASGR_NUM>118103</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>116604</ALIGHT_PASGR_NUM>
<WORK_DT>20130723</WORK_DT>
<COMMT/>
</row>
<row>
<USE_MON>201306</USE_MON>
<LINE_NUM>중앙선</LINE_NUM>
<SUB_STA_NM>오빈</SUB_STA_NM>
<RIDE_PASGR_NUM>10590</RIDE_PASGR_NUM>
<ALIGHT_PASGR_NUM>10020</ALIGHT_PASGR_NUM>
<WORK_DT>2013072